# Environment

### Env with **Pytorch 1.6** 

# Dataset Overview (See 3.)

In [9]:
import pandas as pd

In [13]:
video_info_df = pd.read_csv("demo/video_info.csv", index_col=0)
video_info_df.head()

,filename,class,width,height,frames_per_second,num_frames
0,April_09_brush_hair_u_nm_np1_ba_goo_0.avi,brush_hair,320,240,30.0,409
1,April_09_brush_hair_u_nm_np1_ba_goo_1.avi,brush_hair,320,240,30.0,395
2,April_09_brush_hair_u_nm_np1_ba_goo_2.avi,brush_hair,320,240,30.0,323
3,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair,416,240,30.0,246
4,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair,416,240,30.0,159


In [14]:
video_info_df['class'].unique()

array(['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb',
       'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat',
       'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit',
       'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick',
       'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike',
       'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow',
       'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault',
       'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk',
       'throw', 'turn', 'walk', 'wave'], dtype=object)

In [18]:
video_info_df[video_info_df['class']=='ride_bike'].index[:10]

Int64Index([3544, 3545, 3546, 3547, 3548, 3549, 3550, 3551, 3552, 3553], dtype='int64')

# Dataset

In [1]:
import os
os.getcwd()

'/datasets/test/dev/try_module_vision/detectron2_action_recognition'

In [2]:
BASE_PTH = "../"*(len(os.getcwd().split("/"))-1)  #"../../../../../"
DATASET_PTH = BASE_PTH+"datasets/public/HMDB51"

In [3]:
import glob
filenames = sorted(glob.glob(DATASET_PTH+"/*/*.avi"))
len(filenames), filenames[:2]

(6766,
 ['../../../../../datasets/public/HMDB51/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0.avi',
  '../../../../../datasets/public/HMDB51/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1.avi'])

In [4]:
from utils import display_video

i=
- 1100 : [easy] 一人運球
- 600: [easy] 一人攀岩
- 300: [meduim] 一人踢球　一人接球
- 800: [meduim] 一人高空跳下　偵測不佳
- 190 : [hard] 後空翻　偵測不佳

In [19]:
i = 3544

In [20]:
display_video(filenames[i])

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4

Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [21]:
from video_human_kp_detect import *
import numpy as np

# Video2KP

In [22]:
predictor, cfg = prepare_predictor(THRESH=0.9)
visualizer = prepare_video_visualizer(cfg)

In [23]:
%%time
predict_single_video(predictor, visualizer, 
                     VIDEO_FILE_PTH=filenames[i], 
                     INFO_OUTPUT_PTH="demo/info_dict.joblib", 
                     save_video=True, VIDEO_OUTPUT_PTH="demo/out.avi", 
                     top_k_selected_roles=1,
                     save_video_selected_roles=True, VIDEO_OUTPUT_PTH_SELECTED_ROLES="demo/out_s.avi",
                     tracking_max_distance_limit=[27,50],
                     print_status = True,
                    )

收集 video 內角色的關鍵點..


 99%|█████████▉| 80/81 [00:04<00:00, 16.88it/s]

視覺化關鍵點 並輸出video..



 99%|█████████▉| 80/81 [00:02<00:00, 31.56it/s]


save video [demo/out.avi]
選出 移動最多/最頻繁出現/最完整 的角色 k 個..
  角色[0]出現在 frame [0]
所有角色 frequency_score : [0.81481481]
所有角色 completeness_score : [0.81481481]
所有角色 motion_score : [4.07944719]
selected_roles(由分數大到小):  [0]
  selected角色 frequency_score : [0.81481481]
  selected角色 completeness_score : [0.81481481]
  selected角色 motion_score : [4.07944719]
重新篩選出這 k 個角色的關鍵點資訊..
視覺化k 個角色的關鍵點 並輸出video..


 99%|█████████▉| 80/81 [00:02<00:00, 31.18it/s]


save video [demo/out_s.avi]
save video_dict [demo/info_dict.joblib]
CPU times: user 9.46 s, sys: 570 ms, total: 10 s
Wall time: 10 s


In [24]:
display_video("demo/out.avi")

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [25]:
display_video("demo/out_s.avi")

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [26]:
import joblib
info_d = joblib.load("demo/info_dict.joblib")

In [27]:
info_d.keys()

dict_keys(['width', 'height', 'frames_per_second', 'num_frames', 'num_role_register', 'boxes_buffer', 'kps_buffer', 'roles_buffer', 'frames_split_pos', 'cost_matrix_collection', 'image_size', 'bbox_area_by_role', 'num_frames_by_role', 'motion_distance_by_role', 'frames_by_role', 'completeness_by_role', 'selected_roles', 'boxes_buffer_selected', 'kps_buffer_selected'])

In [28]:
n_frames = info_d['num_frames_by_role']
print(f"影片中有 [{len(n_frames)}] 個人,分別出現的frame數為 {n_frames}")
print(f"選擇的角色id為 { info_d['selected_roles']}")

影片中有 [1] 個人,分別出現的frame數為 [66.]
選擇的角色id為 [0]
